In [1]:
from pathlib import Path
import json
import os

from rdflib import URIRef, Namespace

from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Template
from buildingmotif.namespaces import BRICK
from buildingmotif.semantic_graph_synthesizer.bindings_utils import unify_bindings, evaluate_bindings
from buildingmotif.semantic_graph_synthesizer.classes import TokenizedLabel
from buildingmotif.semantic_graph_synthesizer.semantic_graph_synthesizer import SemanticGraphSynthesizer
from buildingmotif.semantic_graph_synthesizer.bipartite_token_mapper import BipartiteTokenMapper
from buildingmotif.semantic_graph_synthesizer.template_synthesizer import BrickTemplateSynthesizer

In [2]:
bm = BuildingMOTIF("sqlite://")
brick = Library.load(ontology_graph="https://brickschema.org/schema/1.4.1/Brick.ttl")
lib = Library.create("mylib")

/Users/gabe/src/NREL/BuildingMOTIF/.venv/lib/python3.11/site-packages/pyshacl/extras/__init__.py:46: Warning: Extra "js" is not satisfied because requirement pyduktape2 is not installed.
  warn(Warning(f"Extra \"{extra_name}\" is not satisfied because requirement {req} is not installed."))
2024-09-03 12:59:13,866 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/prefix (No row was found when one was required). Check this is loaded into BuildingMOTIF
2024-09-03 12:59:13,867 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/schema/shacl/qudt (No row was found when one was required). Check this is loaded into BuildingMOTIF
2024-09-03 12:59:13,867 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/quantitykind (No row was found when one was required). Check this is loaded into BuildingMOTIF
2024-09-03 12:59:13,868 | root |  WARNING: An ontology could not resolve a dependency 

shape https://w3id.org/rec#Lease has dependency https://w3id.org/rec#Agent
shape https://w3id.org/rec#Lease has dependency https://w3id.org/rec#Agent
shape https://w3id.org/rec#ActuationEvent has dependency https://brickschema.org/schema/Brick#Point
shape https://w3id.org/rec#Premises has dependency https://w3id.org/rec#Architecture
shape https://brickschema.org/schema/Brick#Loop has dependency https://brickschema.org/schema/Brick#Equipment
shape https://w3id.org/rec#FurnitureCollection has dependency https://w3id.org/rec#Furniture
shape https://w3id.org/rec#EquipmentCollection has dependency https://brickschema.org/schema/Brick#Equipment
shape https://w3id.org/rec#ObservationEvent has dependency https://brickschema.org/schema/Brick#Point
shape https://brickschema.org/schema/Brick/ref#IFCReference has dependency https://brickschema.org/schema/Brick/ref#ifcProject
shape https://w3id.org/rec#ExceptionEvent has dependency https://brickschema.org/schema/Brick#Point
shape https://w3id.org/r

2024-09-03 12:59:15,351 | root |  WARNING: Warning: could not find dependee n90e6ff708c8f45ac96d2987ac2339b84b7697 in libraries []
2024-09-03 12:59:15,353 | root |  WARNING: Warning: could not find dependee n90e6ff708c8f45ac96d2987ac2339b84b7703 in libraries []
2024-09-03 12:59:15,355 | root |  WARNING: Warning: could not find dependee n90e6ff708c8f45ac96d2987ac2339b84b7699 in libraries []
2024-09-03 12:59:15,357 | root |  WARNING: Warning: could not find dependee n90e6ff708c8f45ac96d2987ac2339b84b7701 in libraries []
2024-09-03 12:59:15,359 | root |  WARNING: Warning: could not find dependee n90e6ff708c8f45ac96d2987ac2339b84b7705 in libraries []
2024-09-03 12:59:15,362 | root |  WARNING: Warning: could not find dependee n90e6ff708c8f45ac96d2987ac2339b84b7621 in libraries []
2024-09-03 12:59:15,364 | root |  WARNING: Warning: could not find dependee n90e6ff708c8f45ac96d2987ac2339b84b7619 in libraries []
2024-09-03 12:59:15,366 | root |  WARNING: Warning: could not find dependee n90e6ff

In [3]:
syn = BrickTemplateSynthesizer(brick, dest_library=lib)

In [4]:
%load_ext snakeviz

In [5]:
t = syn.generate_graphs(BRICK.AHU, 3)

[rdflib.term.URIRef('urn:___param___#name'), rdflib.term.URIRef('urn:___param___#p1')]
@prefix P: <urn:___param___#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .

P:name a brick:AHU ;
    brick:isMeteredBy P:p1 .

P:p1 a brick:Meter .


[rdflib.term.URIRef('urn:___param___#name'), rdflib.term.URIRef('urn:___param___#p2')]
@prefix P: <urn:___param___#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .

P:name a brick:AHU ;
    brick:hasPart P:p2 .

P:p2 a brick:Equipment .


[rdflib.term.URIRef('urn:___param___#name'), rdflib.term.URIRef('urn:___param___#p3')]
@prefix P: <urn:___param___#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix ns1: <https://w3id.org/rec#> .

P:name a brick:AHU ;
    ns1:hasPoint P:p3 .

P:p3 a brick:Point .


[rdflib.term.URIRef('urn:___param___#name'), rdflib.term.URIRef('urn:___param___#p4')]
@prefix P: <urn:___param___#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .

P:name a brick:AHU ;
    brick:hasPo

In [6]:
syn.write_templates(t)

In [7]:
t[0].body.serialize()

'@prefix P: <urn:___param___#> .\n@prefix brick: <https://brickschema.org/schema/Brick#> .\n\nP:name a brick:AHU ;\n    brick:isMeteredBy P:p1 .\n\nP:p1 a brick:Meter ;\n    brick:isMeteredBy P:p5 .\n\nP:p5 a brick:Meter .\n\n'

In [8]:
print(t[-2].graph.serialize())

AttributeError: 'Template' object has no attribute 'graph'

In [11]:
l = Library.load(directory="templates")

In [12]:
l.get_templates()

[Template(_id=1729, _name='autogen_template_AHU_0', body=<Graph identifier=fa7a33ea-128f-4fc8-a642-f178a2a2a29e (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x10ed76650>),
 Template(_id=1730, _name='autogen_template_AHU_1', body=<Graph identifier=60c1031e-0355-4f65-9ff0-de68d6d9fb4d (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x10ed76650>),
 Template(_id=1731, _name='autogen_template_AHU_10', body=<Graph identifier=097a4f26-d0ce-443d-a2db-47fa15d287d6 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x10ed76650>),
 Template(_id=1732, _name='autogen_template_AHU_11', body=<Graph identifier=95ef2eb2-a5b4-4265-971f-b3cee7bc9fd1 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x10ed766